# Как сделать "2048" #

Импортируем все нужные библиотеки

In [1]:
import numpy as np
import random
import arcade as ar
import json

Создаем функции для сохранения поля в файле, а также его воспроизведения при повторном запуске игры

In [2]:
def load():
    try:
        with open("version.json", "r") as file:
            version = json.load(file)
            return np.array(version)
    except FileNotFoundError:
        return []

def save(version):
    with open("version.json", "w") as file:
        json.dump(version.tolist(), file)

Создаем функции, которые:
* Проверяют наличие пустых мест
* Добавляют 2 (с 80% вероятностью) или 4 (20%) в случайное пустое место

In [3]:
def test (mas):
    empt = []
    for i in range (4):
        for j in range (4):
            if mas[i][j] == 0:
                num = number(i,j)
                empt.append(num)
    return empt

def number (i,j):
    return i*4 + j + 1

def index (num):
    num-=1
    x,y = num//4, num%4
    return x,y

def insert(mas):
    empty = test(mas)
    n = random.choice(empty)
    x,y = index(n)
    if np.random.sample() <= 0.8:
        mas[x][y] = 2
    else:
        mas[x][y] = 4
    return(mas)

Создаем пустую таблицу

Перед началом игры проверяем наличие сохранения:
* Запускаем сохраненную игру
* Или начинаем новую

Создаем словарь, который будет соотносить значение ячейки и ее цвет

In [4]:
start = np.zeros((4,4), dtype=np.int)

version = load()
if version != []:
    game = version
else:
    game = insert(np.copy(start))
    
d_color = {
    2: ar.csscolor.DEEP_PINK,
    4: ar.csscolor.CYAN,
    8: ar.csscolor.DEEP_SKY_BLUE,
    16: ar.csscolor.MEDIUM_SPRING_GREEN,
    32: ar.csscolor.DARK_VIOLET,
    64: ar.csscolor.LIME,
    128: ar.csscolor.BLUE,
    256: ar.csscolor.MAGENTA,
    512: ar.csscolor.SILVER,
    1024: ar.csscolor.GOLD,
    2048: ar.csscolor.DARK_RED
}

Создаем функции для сдвига ячеек в матрице

Также в этом блоке функция проверки возможности сдвига

In [5]:
def shift(st):
    a = []
    for i in range(4):
        if st[i] == 0:
            a.append(i)
    st = np.delete(st,a)
    st = np.append(st, np.zeros(4 - len(st),dtype=np.int))
    for i in range(3):
        if st[i] == st[i+1]:
            st = np.delete(st, i+1)
            st[i] *= 2
            st = np.append(st, 0)
    return st

def left(cop):
    for i in range(4):
        cop[i] = shift(cop[i])
    return cop

def right(cop):
    for i in range(4):
        cop[i] = shift(cop[i][::-1])
        cop[i] = np.flip(cop[i]) 
    return cop

def up(cop):
    cop = np.transpose(cop)
    cop = left(cop)
    return np.transpose(cop)

def down(cop):
    cop = np.transpose(cop)
    cop = right(cop)
    return np.transpose(cop)

def check(game, function):
    attempt = np.copy(game)
    function(attempt)
    if not np.array_equal(attempt, game):
        return(1)
    else:
        return(0)

Принцип самой игры

Управление:
* "Space" - начать новую игру
* "Escape" - выйти из игры
* "Up" - совершить сдвиг вверх
* "Down" - совершить сдвиг вниз
* "Left" - совершить сдвиг влево
* "Right" - совершить сдвиг вправо

Каждый ход проверяется наибольшее значение для проверки достижения выигрышного состояния

Каждый ход таблица сохраняется в файл

Если в таблице есть 2048 - игрок победил

Если нельзя совершить сдвиг ни в одну из сторон - игрок проиграл

Цикл игры (ход):
1. Программа проверяет возможность сдвига в одну из сторон (иначе - игра окончена)
2. Игрок выбирает сторону сдвига, ячейки сдвигаются
3. При наличии пустого места, добавляется новая 2(4)
4. Отрисовывается новое состояние игрового поля (если есть 2048 - победа)

In [6]:
class MyGame(ar.Window):
    def __init__(self, width, height, title):
        super().__init__(width, height, title)
        self.score = 0
        self.death = 0
        self.game = game
        
    class square:
        def __init__(self, x1, x2, y2, y1, color):
            self.x1 = x1
            self.x2 = x2
            self.y2 = y2
            self.y1 = y1
            self.color = color
    
        def draw(self):
            ar.draw_lrtb_rectangle_filled(self.x1, self.x2, self.y2, self.y1, self.color) 
    
    def setup(self):
        ar.set_background_color(ar.color.LICORICE)
    
    def on_key_press(self, key, modifiers):
        
        if key == ar.key.ESCAPE:
            ar.close_window()
            
        if key == ar.key.SPACE:
            self.game = insert(np.copy(start))
            self.score = 0
            self.death = 0
            
        if key == ar.key.LEFT:
            cop = np.copy(self.game)
            left(self.game)
            if not np.array_equal(cop, self.game):
                insert(self.game)
                     
        if key == ar.key.RIGHT:
            cop = np.copy(self.game)
            right(self.game)
            if not np.array_equal(cop, self.game):
                insert(self.game)
        
        if key == ar.key.UP:
            cop = np.copy(self.game)
            up(self.game)
            if not np.array_equal(cop, self.game):
                insert(self.game)
                    
        if key == ar.key.DOWN:
            cop = np.copy(self.game)
            down(self.game)
            if not np.array_equal(cop, self.game):
                insert(self.game)
                
    def on_update(self, delta_time):
        direct = 0
        direct += check(self.game, left)
        direct += check(self.game, right)
        direct += check(self.game, up)
        direct += check(self.game, down)
        if direct == 0:
            self.death = 1
        save(self.game)
            
    def on_draw(self):
        ar.start_render()
        ar.draw_lrtb_rectangle_filled(100, 500, 500, 100, ar.csscolor.TURQUOISE)
        ar.draw_rectangle_filled(100, 300, 10, 410, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(200, 300, 10, 400, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(300, 300, 10, 400, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(400, 300, 10, 400, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(500, 300, 10, 410, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(300, 100, 400, 10, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(300, 200, 400, 10, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(300, 300, 400, 10, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(300, 400, 400, 10, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_rectangle_filled(300, 500, 400, 10, ar.csscolor.MEDIUM_TURQUOISE)
        ar.draw_text("It's 2048 time", 150, 530, ar.color.WHITE, 24)
        self.score = np.max(self.game)
        x1 = 105
        y1 = 405
        x2 = 195
        y2 = 495
        for i in range (4):
            for j in range (4):
                if self.game[i][j]:
                    a = self.game[i][j]
                    color = d_color[a]
                    self.square(x1,x2,y2,y1,color).draw()
                    text = str(a)
                    ar.draw_text(text, x1+10, y1+25, ar.color.BLACK, 24, align="center")
                x1 += 100
                x2 += 100
            x1 = 105
            x2 = 195
            y1 -= 100
            y2 -= 100
        if self.score == 2048:
            ar.draw_text("ВЫ ПОБЕДИЛИ!!!", 0, 250, ar.color.WHITE, 64)
        elif self.death == 1:
            ar.draw_text("ВЫ ПРОИГРАЛИ:/", 0, 250, ar.color.WHITE, 64)        

Запускаем игру

In [7]:
twenty48 = MyGame(600, 600, 2048)
ar.finish_render()
ar.run()